In [ ]:
pip install langchain langchain_community langchain_groq sentence_transformers chromadb faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.3 MB/s eta 0

In [ ]:
pip install posthog

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
loaders = [
    TextLoader("/content/paul_graham_essay.txt"),
    TextLoader("/content/state_of_the_union.txt"),
]

In [ ]:
docs = []
for loader in loaders:
  docs.extend(loader.load())

len(docs)

2

In [ ]:
for i, doc in enumerate(docs):
    print(f"Document {i+1} - Characters: {len(doc.page_content)}")

Document 1 - Characters: 75012
Document 2 - Characters: 38540


In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

In [ ]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
from langchain.storage import InMemoryStore
from langchain.vectorstores import Chroma

In [ ]:
import os
os.environ["CHROMA_TELEMETRY"] = "False"

import logging
logging.getLogger("chromadb").setLevel(logging.CRITICAL)

In [ ]:
vector_store = Chroma(
    collection_name="full_douments", embedding_function= embedding
)

In [ ]:
store = InMemoryStore()

In [ ]:
from langchain.retrievers import ParentDocumentRetriever
retriever = ParentDocumentRetriever(
    vectorstore=vector_store,
    docstore=store,
    child_splitter= splitter
)

In [ ]:
retriever.add_documents(docs, ids=None)

In [ ]:
len(list(store.yield_keys()))

2

In [ ]:
response = retriever.invoke("What did the president say about Ketanji Brown Jackson?")
len(response[0].page_content)

38540

In [ ]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

In [ ]:
vector_store = Chroma(
    collection_name="parent_documents", embedding_function= embedding
)
store = InMemoryStore()

In [ ]:
retriever = ParentDocumentRetriever(
    vectorstore=vector_store,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [ ]:
retriever.add_documents(docs)

In [ ]:
len(list(store.yield_keys()))

66

In [ ]:
response = retriever.invoke("What did the president say about Ketanji Brown Jackson?")
response[0].page_content

'In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections. \n\nWe cannot let this happen. \n\nTonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. \n\nA former top litigator in private practice. A former 

In [ ]:
response[1].page_content

"At the time I didn't understand what he meant, but gradually it dawned on me that he was saying I should quit. This seemed strange advice, because YC was doing great. But if there was one thing rarer than Rtm offering advice, it was Rtm being wrong. So this set me thinking. It was true that on my current trajectory, YC would be the last thing I did, because it was only taking up more of my attention. It had already eaten Arc, and was in the process of eating essays too. Either YC was my life's work or I'd have to leave eventually. And it wasn't, so I would.\n\nIn the summer of 2012 my mother had a stroke, and the cause turned out to be a blood clot caused by colon cancer. The stroke destroyed her balance, and she was put in a nursing home, but she really wanted to get out of it and back to her house, and my sister and I were determined to help her do it. I used to fly up to Oregon to visit her regularly, and I had a lot of time to think on those flights. On one of them I realized I wa

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
from google.colab import userdata
import os
GROQ_API_KEY= userdata.get('GROQ_API_KEY')
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [ ]:
llm = ChatGroq(model = "gemma2-9b-it")

In [ ]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson?"
result = chain({"query": query})
result["result"]

'The president said Ketanji Brown Jackson is one of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. \n\nHe also said she is a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers. He noted that she is a consensus builder and has received broad range of support since her nomination, from the Fraternal Order of Police to former judges appointed by Democrats and Republicans.  \n'